In [1]:
import pandas as pd
import gensim
import json

/home/eamonn/.local/lib/python3.5/site-packages/gensim/utils.py:1015: UserWarning: Pattern library is not installed, lemmatization won't be available.
  warnings.warn("Pattern library is not installed, lemmatization won't be available.")


In [2]:
fns = ['/home/eamonn/Projects/corpuscule/ycac/csv/ycac_lite.corpus']

In [3]:
df = pd.DataFrame()

In [4]:
for fn in fns:
    df = df.append(pd.read_csv(fn))

In [5]:
len(df)

2783459

In [6]:
def str_to_vec(s):
    l = eval(s)
    rep = ""
    for i in range(12):
        if i in l:
            rep += "1"
        else:
            rep += "0"
    return rep

In [7]:
df.head(5)

,offset,PCsInNormalForm,file,Composer
0,0.000000,"[2, 6, 9]",Allegro HWV 323 D Major.mid,Handel
1,0.625000,"[2, 6]",Allegro HWV 323 D Major.mid,Handel
2,1.125000,"[1, 2, 6, 9]",Allegro HWV 323 D Major.mid,Handel
3,1.333333,"[7, 11]",Allegro HWV 323 D Major.mid,Handel
4,1.375000,"[7, 9, 11, 1]",Allegro HWV 323 D Major.mid,Handel


In [8]:
df['canonical'] = df.PCsInNormalForm.apply(str_to_vec)

In [11]:
df.head(5)

,offset,PCsInNormalForm,file,Composer,canonical
0,0.000000,"[2, 6, 9]",Allegro HWV 323 D Major.mid,Handel,001000100100
1,0.625000,"[2, 6]",Allegro HWV 323 D Major.mid,Handel,001000100000
2,1.125000,"[1, 2, 6, 9]",Allegro HWV 323 D Major.mid,Handel,011000100100
3,1.333333,"[7, 11]",Allegro HWV 323 D Major.mid,Handel,000000010001
4,1.375000,"[7, 9, 11, 1]",Allegro HWV 323 D Major.mid,Handel,010000010101


In [9]:
u = df.file.unique()
len(u)

2048

In [12]:
df.head(10).file[1]

'Allegro HWV 323 D Major.mid'

In [13]:
df.to_csv('all_canonical.csv')

In [16]:
meta = pd.read_csv('/home/eamonn/Projects/corpuscule/ycac/csv/metadata/YCAC-metadata.csv')

CParserError: Error tokenizing data. C error: Buffer overflow caught - possible malformed input file.


In [43]:
meta.Date.fillna(meta.Range, inplace=True)

In [49]:
import numpy as np

def process_range(d):
    try:
        if '-' in d:
            a = int(d.split('-')[0])
            b = int(d.split('-')[1])
            return (a + b) / 2
        else:
            return int(d)
    except:
        return np.nan

In [50]:
meta['EstDate'] = meta.Date.apply(process_range)

In [53]:
df['file'] = df.file.apply(lambda s: s.replace('.mid', ''))
df['Filename'] = df['file']

In [54]:
master = pd.merge(df, meta, on='Filename')

In [55]:
master.to_csv('all_canonical.csv')

In [17]:
master = pd.read_csv('all_canonical.csv', dtype={'canonical':object})

## Dump to json

In [19]:
composers = list(master.Composer.value_counts().keys()[:20])

In [20]:
top_20_comp

['Mozart', 'Haydn', 'Bach', 'Handel', 'Casadesus']

In [1]:
# dump per composer


import json

for composer in top_20_comp:
    sub = master[master.Composer_x == composer]
    sub = sub[['file', 'canonical']]

    files = list(sub.file)
    vecs = list(sub.canonical)

    fv = pd.DataFrame(zip(files,vecs),columns = ['file', 'canonical'])
    fv.reset_index()
    
    done = fv.groupby('file')['canonical'].apply(list)
    docs = list(done)

    print("{} group done".format(composer))

    with open('{}.json'.format(composer), 'w') as f:
        json.dump(docs, f)

    print("{} dump done".format(composer))

NameError: name 'top_20_comp' is not defined

In [22]:
# dump whole corpus

sub = master
sub = sub[['file', 'canonical']]

files = list(sub.file)
vecs = list(sub.canonical)

fv = pd.DataFrame(list(zip(files,vecs)),columns = ['file', 'canonical'])
fv.reset_index()

done = fv.groupby('file')['canonical'].apply(list)
docs = list(done)

composer = 'ycac'

print("{} group done".format(composer))

with open('{}.json'.format(composer), 'w') as f:
    json.dump(docs, f)

print("{} dump done".format(composer))

ycac group done
ycac dump done


In [3]:
eras = [
    (1650, 1699),
    (1700, 1749),
    (1750, 1799),
    (1800, 1849),
    (1850, 1899),
    (1900, 1950)
]

In [ ]:
for era in eras:
    start_year = era[0]
    end_year = era[1]
    
    sub = master[master.EstDate.isin(range(start_year, end_year))]
    sub = sub[['file', 'canonical']]

    label = "{}_to_{}".format(start_year, end_year)
    
    files = list(sub.file)
    vecs = list(sub.canonical)

    fv = pd.DataFrame(zip(files,vecs),columns = ['file', 'canonical'])
    fv.reset_index()
    
    done = fv.groupby('file')['canonical'].apply(list)
    docs = list(done)

    print("{} group done".format(label))

    with open('{}.json'.format(label), 'w') as f:
        json.dump(docs, f)

    print("{} dump done".format(label))

1650_to_1699 group done
1650_to_1699 dump done
1700_to_1749 group done
1700_to_1749 dump done